# A BLAS-LAPACK benchmark

Taken from [SciPy](https://github.com/scipy/scipy/blob/main/benchmarks/benchmarks/blas_lapack.py).

First, do some initialization:

In [2]:
import numpy as np
import random

np.random.seed(1234)
random.seed(1234)

Then, define the relevant parts of the the common utility file:

In [3]:
"""
Airspeed Velocity benchmark utilities
"""
import os


class Benchmark:
    """
    Base class with sensible options
    """
    pass


class safe_import:

    def __enter__(self):
        self.error = False
        return self

    def __exit__(self, type_, value, traceback):
        if type_ is not None:
            self.error = True
            suppress = not (
                os.getenv('SCIPY_ALLOW_BENCH_IMPORT_ERRORS', '1').lower() in
                ('0', 'false') or not issubclass(type_, ImportError))
            return suppress

And here is the definition of the actual benchmark:

In [4]:
import numpy as np

with safe_import():
    import scipy.linalg.blas as bla


class GetBlasLapackFuncs(Benchmark):
    """
    Test the speed of grabbing the correct BLAS/LAPACK routine flavor.
    In particular, upon receiving strange dtype arrays the results shouldn't
    diverge too much. Hence the results here should be comparable
    """

    param_names = ['dtype1', 'dtype2',
                   'dtype1_ord', 'dtype2_ord',
                   'size']
    params = [
        ['b', 'G', 'd'],
        ['d', 'F', '?'],
        ['C', 'F'],
        ['C', 'F'],
        [10, 100, 1000]
    ]

    def setup(self, dtype1, dtype2, dtype1_ord, dtype2_ord, size):
        self.arr1 = np.empty(size, dtype=dtype1, order=dtype1_ord)
        self.arr2 = np.empty(size, dtype=dtype2, order=dtype2_ord)

    def time_find_best_blas_type(self, dtype1, dtype2, dtype1_ord, dtype2_ord, size):
        prefix, dtype, prefer_fortran = bla.find_best_blas_type((self.arr1, self.arr2))

Now, let's run it:

In [12]:
benchmark = GetBlasLapackFuncs()
benchmark.setup(np.int32, np.int32, 'C', 'C', 8)

for i in range(100000000):
    benchmark.time_find_best_blas_type(np.int32, np.int32, 'C', 'C', 5)